In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [2]:
data = pd.read_csv('/content/drive/MyDrive/stamatics-titanic/stamatics-titanic/train.csv')
test_data = pd.read_csv('/content/drive/MyDrive/stamatics-titanic/stamatics-titanic/test.csv')

data.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
print(data.columns)

cols_to_use = ['Sex', 'Age','Fare']
numerical_cols = [col for col in cols_to_use if data[col].dtype in ['int64', 'float64']]
categorical_cols = [col for col in cols_to_use if data[col].dtype == 'object' and data[col].nunique()<10]

test_set = test_data[cols_to_use]

print(f'numerical_cols: {numerical_cols}')
print(f'categorical_cols: {categorical_cols}')

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')
numerical_cols: ['Age', 'Fare']
categorical_cols: ['Sex']


In [4]:
X = data[cols_to_use]
y = data.Survived


X_train, X_valid, y_train, y_valid = train_test_split(X,y, train_size = 0.8, test_size = 0.2, random_state = 1)

In [5]:
X_train = pd.get_dummies(X_train)
X_valid = pd.get_dummies(X_valid)
test_set = pd.get_dummies(test_set)
X_train, X_valid = X_train.align(X_valid, join='left', axis=1)
X_train, test_set = X_train.align(test_set, join='left', axis=1)

In [6]:
print(test_set.isnull().sum())
impute = SimpleImputer(strategy = 'mean')
array = test_set['Age'].values.reshape(-1, 1)
test_set['Age'] = impute.fit_transform(array)
array = test_set['Fare'].values.reshape(-1, 1)
test_set['Fare'] = impute.fit_transform(array)
print(test_set.columns)

Age           86
Fare           1
Sex_female     0
Sex_male       0
dtype: int64
Index(['Age', 'Fare', 'Sex_female', 'Sex_male'], dtype='object')


In [7]:
mymodel = XGBRegressor(n_estimators = 1000, learning_rate = 0.05, early_stopping_rounds = 5, eval_set = [(X_valid, y_valid)])

numerical_transformer = SimpleImputer(strategy = 'mean')

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output = False))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

pipe = Pipeline(steps = [
    ('preprocessor', preprocessor),
    ('model', XGBRegressor(n_estimators = 1000, learning_rate = 0.05, early_stopping_rounds = 5, eval_set = [(X_valid, y_valid)])),
])

In [8]:
print(X_valid.isnull().sum())

imputer = SimpleImputer(strategy = 'median')
input_array1 = X_train['Age'].values.reshape(-1, 1)
input_array2 = X_valid['Age'].values.reshape(-1, 1)

X_train['Age'] = imputer.fit_transform(input_array1)
X_valid['Age'] =  imputer.transform(input_array2)



print(X_valid.isnull().sum())

Age           33
Fare           0
Sex_female     0
Sex_male       0
dtype: int64
Age           0
Fare          0
Sex_female    0
Sex_male      0
dtype: int64


In [9]:
eval_set = [(X_valid, y_valid)]
pipe.set_params(model__eval_set=eval_set)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', SimpleImputer(),
                                                  ['Age', 'Fare']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['Sex'])])),
                ('model',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              colsample_bylevel=None...
                              feature_types=None, gamma=None, gpu_id=None,
                              grow_policy=None, importance_type=None,
                              interaction_constraints=None, learning_rate=0.05,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, n_estimators=1000,
                              n_jobs=None, num_parallel_tree=None,
                              predictor=None, ...))])

In [10]:
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import cross_val_score
# pipe.fit(X_train, y_train)
mymodel = XGBClassifier(n_estimators = 1000, learning_rate = 0.01, early_stopping_rounds = 10, eval_set = [(X_valid, y_valid)])
mymodel.fit(X_train, y_train, eval_set = [(X_valid, y_valid)])
y_prediction = mymodel.predict(X_valid)

mae = mean_absolute_error(y_valid, y_prediction)

print(accuracy_score(y_valid, y_prediction))

[07:44:10] WARNING: ../src/learner.cc:767: 
Parameters: { "eval_set" } are not used.

[0]	validation_0-logloss:0.68933
[1]	validation_0-logloss:0.68546
[2]	validation_0-logloss:0.68169
[3]	validation_0-logloss:0.67801
[4]	validation_0-logloss:0.67440
[5]	validation_0-logloss:0.67086
[6]	validation_0-logloss:0.66740
[7]	validation_0-logloss:0.66401
[8]	validation_0-logloss:0.66069
[9]	validation_0-logloss:0.65747
[10]	validation_0-logloss:0.65431
[11]	validation_0-logloss:0.65117
[12]	validation_0-logloss:0.64816
[13]	validation_0-logloss:0.64519
[14]	validation_0-logloss:0.64230
[15]	validation_0-logloss:0.63947
[16]	validation_0-logloss:0.63662
[17]	validation_0-logloss:0.63390
[18]	validation_0-logloss:0.63122
[19]	validation_0-logloss:0.62861
[20]	validation_0-logloss:0.62605
[21]	validation_0-logloss:0.62347
[22]	validation_0-logloss:0.62101
[23]	validation_0-logloss:0.61861
[24]	validation_0-logloss:0.61623
[25]	validation_0-logloss:0.61392
[26]	validation_0-logloss:0.61164
[27]	v

In [11]:
predictions = mymodel.predict(test_set)
ids = test_data['PassengerId']
output = pd.DataFrame({ 'PassengerId' : ids, 'Survived': predictions })
output.to_csv('submission.csv', index=False)